## `1_scoping.ipynb`

### Try adding your own evaluation examples and / or building additional evaluators

N / A

## 2_research_agent.ipynb

### 1. Async Search Implementation for Parallelization

One significant improvement is converting our search tool to use async operations for parallel execution. This dramatically reduces latency when performing multiple searches. Here's how to implement it based on the [reference implementation](https://github.com/langchain-ai/deep_research_from_scratch/blob/dbf54264b65003fc2dca250bac9d0b9b2ea3b03b/src/deep_research_from_scratch/utils.py):

#### Current Synchronous Approach
Our current implementation processes searches sequentially:

```python
def tavily_search_multiple(search_queries: List[str]) -> List[dict]:
    search_docs = []
    for query in search_queries:  # Sequential execution
        result = tavily_client.search(query)
        search_docs.append(result)
    return search_docs
```

####  Async Implementation for Parallel Execution

```python
import asyncio
from langchain_core.tools import atool

async def tavily_search_multiple_async(
    search_queries: List[str], 
    max_results: int = 3,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = True
) -> List[dict]:
    """
    Perform multiple searches concurrently using async operations.
    
    This approach can reduce total search time from N*search_time to 
    max(search_times) when searches are independent.
    """
    tavily_client = TavilyClient()
    
    async def execute_single_search(query: str) -> dict:
        """Execute a single search operation asynchronously."""
        # Wrap the synchronous Tavily call in a thread executor
        loop = asyncio.get_event_loop()
        return await loop.run_in_executor(
            None,  # Use default thread pool
            lambda: tavily_client.search(
                query,
                max_results=max_results,
                include_raw_content=include_raw_content,
                topic=topic
            )
        )
    
    # Execute all searches concurrently
    search_tasks = [execute_single_search(query) for query in search_queries]
    search_results = await asyncio.gather(*search_tasks)
    
    return search_results

# Async tool definition
@atool(description="Async web search utility with parallel execution")
async def tavily_search_async(
    queries: List[str],
    max_results: Annotated[int, InjectedToolArg] = 3,
    topic: Annotated[Literal["general", "news", "finance"], InjectedToolArg] = "general",
    config: RunnableConfig = None
) -> str:
    """
    Fetches results from Tavily search API with parallel execution.
    
    Performance improvement: For N queries, reduces time from N*T to ~T
    where T is the time for a single search operation.
    """
    # Execute searches concurrently
    search_results = await tavily_search_multiple_async(
        queries,
        max_results=max_results,
        topic=topic,
        include_raw_content=True
    )
    
    # Process results (same as synchronous version)
    unique_results = deduplicate_search_results(search_results)
    summarized_results = process_search_results(unique_results)
    return format_search_output(summarized_results)
```

####  Agent Integration with Async Tools

When using async tools, your agent nodes need to handle async execution:

```python
async def tool_node_async(state: ResearcherState):
    """Tool execution node that handles async tool calls."""
    tool_calls = state["researcher_messages"][-1].tool_calls
    
    # Prepare async tool calls
    async_tasks = []
    for tool_call in tool_calls:
        tool = tools_by_name[tool_call["name"]]
        if hasattr(tool, 'ainvoke'):
            # Use async invocation if available
            task = tool.ainvoke(tool_call["args"])
        else:
            # Fallback to sync in thread executor
            loop = asyncio.get_event_loop()
            task = loop.run_in_executor(None, tool.invoke, tool_call["args"])
        async_tasks.append((tool_call, task))
    
    # Execute all tool calls concurrently
    observations = []
    tool_calls_ordered = []
    
    for tool_call, task in async_tasks:
        observation = await task
        observations.append(observation)
        tool_calls_ordered.append(tool_call)
    
    # Create tool message outputs
    tool_outputs = [
        ToolMessage(
            content=observation,
            name=tool_call["name"], 
            tool_call_id=tool_call["id"]
        ) for observation, tool_call in zip(observations, tool_calls_ordered)
    ]
    
    return {"researcher_messages": tool_outputs}

# Update agent with async nodes
agent_builder = StateGraph(ResearcherState, output_schema=ResearcherOutputState)
agent_builder.add_node("llm_call", llm_call)  
agent_builder.add_node("tool_node", tool_node_async)  # Use async version
agent_builder.add_node("compress_research", compress_research)
```

####  Performance Benefits

**Time Complexity Improvement:**
- **Sequential**: O(n × search_time) where n = number of queries
- **Parallel**: O(max(search_times)) ≈ O(search_time) for independent searches

**Real-world Impact:**
- 3 searches taking 2s each: 6s → ~2s (3x faster)
- 5 searches taking 1.5s each: 7.5s → ~1.5s (5x faster)

**Resource Considerations:**
- Higher memory usage due to concurrent operations
- API rate limiting may still apply
- Network bandwidth shared across parallel requests

####  Error Handling with Async

```python
async def robust_parallel_search(queries: List[str]) -> List[dict]:
    """Parallel search with proper error handling."""
    
    async def safe_search(query: str) -> dict:
        """Execute search with error recovery."""
        try:
            return await execute_single_search(query)
        except Exception as e:
            print(f"Search failed for '{query}': {e}")
            return {"query": query, "results": [], "error": str(e)}
    
    # Execute with error isolation
    search_tasks = [safe_search(query) for query in queries]
    results = await asyncio.gather(*search_tasks, return_exceptions=True)
    
    # Filter successful results
    valid_results = [r for r in results if not isinstance(r, Exception)]
    return valid_results
```

####  When to Use Async vs Sync

**Use Async When:**
- Multiple independent search queries
- I/O-bound operations (API calls, file operations)
- Agent performs many tool calls concurrently
- Time-sensitive research scenarios

**Use Sync When:**
- Single search queries
- Sequential dependencies between searches
- Simpler debugging and development
- API has strict rate limiting

This async approach significantly improves agent performance when conducting comprehensive research that requires multiple independent searches.

### 2. Enhanced Prompt Engineering for Summarization

Our current summarization is basic. We can improve it by focusing on research-specific needs:

```python
RESEARCH_FOCUSED_SUMMARIZATION_PROMPT = """
You are summarizing web content for research purposes. Extract the most valuable information for answering research questions.

FOCUS ON:
- Key facts, statistics, and quantitative data
- Expert opinions and authoritative statements  
- Recent developments and current trends
- Comparative information and rankings
- Methodological details and data sources

EXTRACT:
- 3-5 most important points as bullet points
- Relevant direct quotes with context
- Specific numbers, dates, and statistics
- Source credibility indicators

IGNORE:
- Advertisements and promotional content
- Navigation menus and website metadata
- Unrelated sidebar content
- Generic background information

Webpage content: {webpage_content}
Current date: {date}

Provide a structured summary with clear sections for facts, quotes, and key insights.
"""
```

### 3. Intelligent Model Selection

Different models excel at different tasks. Consider using specialized models:

```python
def get_model_for_task(task_type: str):
    """Select optimal model based on task requirements"""
    if task_type == "summarization":
        # Faster, cheaper model for content summarization
        return init_chat_model("openai:gpt-4.1-mini")
    elif task_type == "research_analysis":
        # More powerful model for complex analysis
        return init_chat_model("openai:gpt-4.1")
    elif task_type == "factual_extraction":
        # Model optimized for accuracy
        return init_chat_model("anthropic:claude-3-sonnet")
    return init_chat_model("openai:gpt-4.1")  # Default
```

### 4. Heuristic Search Result Filtering

Implement quality filters to improve result relevance:

```python
def filter_search_results(results: List[dict], query_terms: List[str]) -> List[dict]:
    """Apply heuristics to improve result quality and relevance"""
    filtered = []
    
    for result in results:
        # Content quality signals
        content_length = len(result.get('content', ''))
        if content_length < 100:  # Too short, likely not informative
            continue
        if content_length > 50000:  # Extremely long, may be spam
            continue
            
        # Title quality indicators
        title = result.get('title', '').lower()
        if title.count('|') > 3:  # Likely spam or poorly formatted
            continue
        if any(spam_word in title for spam_word in ['buy now', 'click here', 'free trial']):
            continue
            
        # Domain credibility (customize based on your domain)
        url = result.get('url', '')
        trusted_domains = ['.edu', '.gov', '.org', 'wikipedia.org', 'reuters.com', 'bbc.com']
        is_trusted = any(domain in url for domain in trusted_domains)
        
        # Relevance scoring
        relevance_score = calculate_relevance_score(result, query_terms)
        
        if relevance_score > 0.3 or is_trusted:  # Keep if relevant or from trusted source
            result['relevance_score'] = relevance_score
            result['is_trusted'] = is_trusted
            filtered.append(result)
    
    # Sort by relevance and trust
    return sorted(filtered, key=lambda x: (x.get('is_trusted', False), x.get('relevance_score', 0)), reverse=True)

def calculate_relevance_score(result: dict, query_terms: List[str]) -> float:
    """Calculate relevance score based on term frequency and positioning"""
    content = (result.get('content', '') + ' ' + result.get('title', '')).lower()
    title = result.get('title', '').lower()
    
    score = 0.0
    for term in query_terms:
        term = term.lower()
        # Higher weight for title matches
        score += title.count(term) * 0.3
        # Standard weight for content matches
        score += content.count(term) * 0.1
        
    # Normalize by content length
    return min(score / len(content) * 1000, 1.0)
```

### 5. Alternative Search APIs

Diversify your search capabilities with multiple APIs:

```python
from langchain_community.tools import DuckDuckGoSearchRun

@tool
def duckduckgo_search(query: str, max_results: int = 5) -> str:
    """Privacy-focused search without API keys or tracking"""
    search = DuckDuckGoSearchRun()
    results = search.run(f"{query} site:edu OR site:org OR site:gov")
    return results

@tool  
def multi_source_search(queries: List[str], max_results: int = 3) -> str:
    """Search across multiple search engines for comprehensive coverage"""
    tavily_results = tavily_search(queries, max_results)
    duckduckgo_results = "\n\n".join([duckduckgo_search(q, max_results) for q in queries])
    
    return f"=== TAVILY RESULTS ===\n{tavily_results}\n\n=== DUCKDUCKGO RESULTS ===\n{duckduckgo_results}"
```

### 5. Advanced Content Processing

Implement more sophisticated content analysis:

```python
def extract_structured_information(content: str, content_type: str) -> dict:
    """Extract structured information based on content type"""
    
    if content_type == "product_review":
        return extract_product_features(content)
    elif content_type == "research_paper":
        return extract_research_findings(content)
    elif content_type == "news_article":
        return extract_news_facts(content)
    else:
        return extract_general_information(content)

def extract_product_features(content: str) -> dict:
    """Extract product-specific information"""
    # Use regex or NER to extract prices, ratings, features
    import re
    
    price_pattern = r'\$\d+\.?\d*'
    rating_pattern = r'(\d+\.?\d*)\s*(?:out of|/)?\s*5\s*stars?'
    
    prices = re.findall(price_pattern, content)
    ratings = re.findall(rating_pattern, content)
    
    return {
        "prices": prices,
        "ratings": ratings,
        "features": extract_feature_list(content)
    }
```

### 6. Evaluation Framework

Set up comprehensive evaluation for your search improvements:

```python
def evaluate_search_quality(search_results: str, ground_truth: dict) -> dict:
    """Evaluate search result quality across multiple dimensions"""
    
    metrics = {}
    
    # Information coverage
    metrics['coverage'] = calculate_information_coverage(search_results, ground_truth['required_info'])
    
    # Source diversity
    metrics['source_diversity'] = calculate_source_diversity(search_results)
    
    # Factual accuracy (if ground truth available)
    if 'facts' in ground_truth:
        metrics['accuracy'] = calculate_factual_accuracy(search_results, ground_truth['facts'])
    
    # Recency of information
    metrics['recency'] = calculate_information_recency(search_results)
    
    return metrics
```

### Exercise: Implementation Challenge

Try implementing these improvements incrementally:

1. **Start with prompt engineering**: Enhance the summarization prompt to focus on research-specific information
2. **Add filtering**: Implement basic heuristic filtering for search results
3. **Experiment with models**: Try different models for summarization and compare performance
4. **Create evaluations**: Build simple evaluation metrics to measure improvement

### Performance Considerations

When implementing these improvements, consider:
- **Latency**: More processing means slower responses
- **Cost**: Advanced models and multiple API calls increase costs  
- **Quality vs Speed**: Find the right balance for your use case
- **Robustness**: Handle API failures and edge cases gracefully

The key is to start simple and gradually add sophistication based on your specific research needs and performance requirements.